# Machine Transliteration
Data : https://github.com/steveash/NETransliteration-COLING2018Hacettepe University Computer Engineering DepartmentAdvisor: Gönenç ErcanSpring 2018 - Fall 2018

In [1]:
import numpy as np

target_language = "russian"
SUB_LENGTH = 5
FREQ = 10

wd_train = "data/wd_"+target_language+"_64"
wd_val = "data/wd_"+target_language+"_16"
wd_test = "data/wd_"+target_language+"_20"
wd_dict = "data/wd_"+target_language+"_dict"
wd_train_cos = "data/wd_"+target_language+"_train_cos"
wd_dict_cos = "data/wd_"+target_language+"_dict_cos"
wd_val_transliterate = "data/wd_"+target_language+"_val_transliterate"

# #1 Substring Dictionary based on Cosine Similarities

#### "wd_russian_64"
falla	ф а л ь я	2
- eng = ^falla\$
- target = ^фалья\$
- freq = 2

#### "wd_russian_dict"
- Cosine Similarity is computed for each english_target substring pair
- Longest substring is 5 letters long
- freq>10

eng | target | cos | freq | eng_freq | target_freq
--- | --- | --- | --- | --- | ---
^f|^ф|0.9263970661836168|16333|16398|18956
^fa|^ф|0.27099723439962903|1405|1418|18956
^fal|^ф|0.08022706766656275|123|124|18956
^fall|^ф|0.0271763212599322|14|14|18956
^f|^фа|0.29115061343884335|1428|16398|1467
^fa|^фа|0.8583560495795993|1238|1418|1467
^fal|^фа|0.2508752846061912|107|124|1467
^f|^фал|0.08375619512792033|117|16398|119
^fa|^фал|0.2848224813304099|117|1418|119
^fal|^фал|0.8808449207803974|107|124|119
^f|^фаль|0.07241921027820716|86|16398|86
^fa|^фаль|0.2462697730708131|86|1418|86
^fal|^фаль|0.774693511983531|80|124|86

In [ ]:
%%time
vectors, vectors_eng, vectors_target = {}, {}, {}
for line in open(wd_train, encoding='utf8'):
    line = line.split()
    eng = "^" + line[0] + "$"
    target = "^" + "".join(line[1:-1]) + "$"
    freq = int(line[-1])
    sub_eng = []
    for i in range(1, len(eng)-1):
        for j in range(i, len(eng)-1):
            if j-i > SUB_LENGTH: continue
            elif i==1:
                if j==len(eng)-2: sub = eng[i-1::]
                else: sub = eng[i-1:j+1]
            elif j==len(eng)-2: sub = eng[i::]
            else: sub = eng[i:j+1]
            sub_eng.append(sub)
            if sub not in vectors_eng: vectors_eng[sub] = freq
            else: vectors_eng[sub] += freq
        
    for i in range(1, len(target)-1):
        for j in range(i, len(target)-1):
            if j-i > SUB_LENGTH: continue
            elif i == 1:
                if j==len(target)-2: sub = target[i-1::]
                else: sub = target[i-1:j+1]
            elif j==len(target)-2: sub = target[i::]
            else: sub = target[i:j+1]
            if sub not in vectors_target: vectors_target[sub] = freq
            else: vectors_target[sub] += freq
            
            for e in sub_eng:
                eng_target = e + "_" + sub
                if eng_target.count("^")!=1 and eng_target.count("$")!=1:
                    if eng_target not in vectors: vectors[eng_target] = freq
                    else: vectors[eng_target] += freq
        
out = open(wd_dict, "w+", encoding="utf8")
for sub, freq in vectors.items():
    eng = vectors_eng.get(sub.split("_")[0])
    target = vectors_target.get(sub.split("_")[1])
    cos = freq/np.sqrt(eng*target)
    if freq>FREQ:
        out.write(sub+","+str(cos)+","+str(freq)+","+str(eng)+","+str(target)+"\n")
out.close()

# Word Partition

`partition_list = partition("falla", len("falla"))`

1 | 2 | 3 | 4 | 5
--- | --- | --- | --- | --- 
^falla\$ | ^fall_a\$ | ^fal_l_a\$ | ^fa_l_l_a\$ | ^f_a_l_l_a\$
- | ^fa_lla\$  | ^fa_ll_a\$ | ^f_al_l_a\$ | -
- | ^fal_la\$ | ^fa_l_la\$ | ^f_a_ll_a\$ | -
- | ^fa_lla\$ | ^f_all_a\$ | ^f_a_l_la\$ | -
- | ^f_alla\$ | ^f_al_la\$ | - | -
- | - | ^f_a_lla\$ | - | -
     
   


In [2]:
def partition(word, max_len):
    partition_list = [[] for i in word]
    matrix = [[] for i in word]
    
    for i in range(0, len(word)):
        for j in range(i, len(word)):
            if j<=i+SUB_LENGTH:
                matrix[i].append(word[i:j+1])    
                    
    stack = [i for i in matrix[0]]
    top = len(stack)-1
    
    while top!=-1:
        temp = stack[top]
        templen = len(temp) - len(temp.split("_"))+1
        if templen == len(word):
            partition_list[len(temp.split("_"))-1].append("^"+temp+"$")
        else:
            for i in matrix[templen]:
                if len(stack) > top:
                    stack[top] = temp + "_" + i
                    top+=1
                else:
                    stack.append(temp + "_" + i)
                    top+=1
        top-=1    
    return partition_list[0:max_len]

# Train

#### "wd_russian_train_cos"
target | english | best split score
--- | --- | ---
^ф\_а\_л\_ья\$|^f\_a\_l\_la\$|0.8732937987793205

In [ ]:
%%time
cos_dict = {}
for line in open(wd_dict, encoding='utf8'):
    line = line.strip().split(",")
    cos_dict[line[0]] = [i for i in map(float,line[1::])]
    
out = open("za", "w+", encoding="utf8")
count = 0
for line in open(wd_train, encoding='utf8'):
    count += 1
    line = line.split()
    eng, target, freq = line[0], "".join(line[1:-1]), int(line[-1])
    best, best_target, best_eng = 0.0, "", ""
    
    max_len = min(len(eng),len(target))
    eng_list = partition(eng,max_len)
    target_list = [i for j in partition(target,max_len) for i in j]
    for t in target_list:
        t_arr = t.split("_")
        for e in eng_list[len(t_arr)-1]:
            score = 0.0
            e_arr = e.split("_")
            for i in range(len(t_arr)):
                key = e_arr[i] + "_" + t_arr[i]
                if key in cos_dict: score += cos_dict.get(key)[0]
        score /= len(t_arr)
        if score > best:
            best = score
            best_eng = e
            best_target = t
    out.write(best_target + "," + best_eng + "," + str(best) + "\n")    
out.close()

# #2 Substring Dictionary based on Trained Data

#### "wd_russian_dict_cos"

- Cosine Similarity is computed for each english_target substring pair
- freq>10

eng | target | cos | freq | eng_freq | target_freq
--- | --- | --- | --- | --- | ---
^f|^ф|0.9472872369467044|1253|1350|1296
a|а|0.9143036189080495|19407|21246|21206
l|л|0.9086893873798547|7987|9019|8566
la\$|ья\$|0.3561381495309969|11|18|53

In [5]:
vectors, vectors_eng, vectors_target = {}, {}, {}
for line in open(wd_train_cos, encoding='utf8'):
    line = line.split(",")
    target = line[0].split("_")
    eng = line[1].split("_")
    score = float(line[2])
    for i in range(len(target)):
        if eng[i] not in vectors_eng: vectors_eng[eng[i]] = 1
        else: vectors_eng[eng[i]] += 1
        if target[i] not in vectors_target: vectors_target[target[i]] = 1
        else: vectors_target[target[i]] += 1
        key = eng[i] + "_" + target[i]
        if key not in vectors: vectors[key] = 1
        else: vectors[key] += 1

out = open(wd_dict_cos, "w+", encoding="utf8")
for sub, freq in vectors.items():
    eng = vectors_eng.get(sub.split("_")[0])
    target = vectors_target.get(sub.split("_")[1])
    cos = freq/np.sqrt(eng*target)
    if freq>FREQ:
        out.write(sub+","+str(cos)+","+str(freq)+","+str(eng)+","+str(target)+"\n")
out.close()

# Test

#### "wd_russian_val_transliterate"

In [6]:
def transliterate(word,target):
    word_list = [i for j in partition(word,len(word)) for i in j]
    best = [0.0,0.0,0.0]
    best_target = [[],[],[]]
    best_word = [[],[],[]]
    for w in word_list:
        w_arr = w.split("_")
        score = 0.0
        match = []
        for p in w_arr:
            if p in target_dict:
                match.append(target_dict.get(p))
                score+=score_dict.get(p)
            else:
                score = 0.0
                break
        score/=len(w_arr)
        minimum = min(best)
        if score>minimum:
            index = best.index(minimum)
            best[index]=score
            best_target[index] = match
            best_word[index] = w_arr
    
    bests = ["".join(i)[1:-1] for i in best_target]
    if bests[0] == target:
        score=1
        out.write("\n + ")
    elif bests[1] == target:
        score=2
        out.write("\n + ")
    elif bests[2] == target:
        score=3
        out.write("\n + ")
    else:
        score=0
        out.write("\n - ")
    out.write(word + "->" + target + "\n________\n")
    [out.write(i + "    ") for i in bests]
    out.write("\n--------\n")
    [out.write("_".join(i)+"    ") for i in best_target]
    out.write("\n")
    [out.write("_".join(i)+"    ") for i in best_word]
    out.write("\n")
    return score

In [14]:
def transliterate_eng_to_target():
    for line in open(wd_dict_cos, encoding='utf8'):
        line = line.strip().split(",")
        eng_target = line[0].split("_")
        if eng_target[0] not in target_dict:
            target_dict[eng_target[0]] = eng_target[1]
            score_dict[eng_target[0]] = float(line[1])
        elif score_dict.get(eng_target[0])<float(line[1]):
            target_dict[eng_target[0]] = eng_target[1]
            score_dict[eng_target[0]] = float(line[1])
    best = [0,0,0,0]
    for line in open(wd_val, encoding='utf8'):
        line = line.split()
        eng = line[0]
        target = "".join(line[1:-1])
        best[transliterate(eng, target)]+=1
    return best
        
def transliterate_target_to_eng():
    for line in open(wd_dict_cos, encoding='utf8'):
        line = line.strip().split(",")
        eng_target = line[0].split("_")
        if eng_target[1] not in target_dict:
            target_dict[eng_target[1]] = eng_target[0]
            score_dict[eng_target[1]] = float(line[1])
        elif score_dict.get(eng_target[1])<float(line[1]):
            target_dict[eng_target[1]] = eng_target[0]
            score_dict[eng_target[1]] = float(line[1])
    best = [0,0,0,0]
    for line in open(wd_val, encoding='utf8'):
        line = line.split()
        eng = line[0]
        target = "".join(line[1:-1])
        best[transliterate(target, eng)]+=1
    return best

In [16]:
target_dict = {}
score_dict = {}
out = open("za", "w+", encoding="utf8")
best = transliterate_eng_to_target()

total = sum(best)
best1 = best[1]
best2 = best[1]+best[2]
best3 = best[1]+best[2]+best[3]

print("Best 1: ", best1/total)
print("Best 2: ", best2/total)
print("Best 3: ", best3/total)
out.close()

Best 1:  0.3360792650520082
Best 2:  0.4361096348037355
Best 3:  0.4421076607698732


# Notes

- While training include 3 best splits into dictionary - scores can also be useful ?
- Most of the tests don't even have 3 splits (insufficient dictionary ? - freq>10 ?)
- At step #1 Extracting freq<10 pairs results one-to-one matches

target | eng
--- | ---
сисвомихарджо | siswomiharjo
^с\_и\_с\_в\_о\_м\_и\_х\_а\_р\_дж\_о\$ | ^s_i_s_w_o_m_i_h_a_r_j_o\$    